In [37]:
#importing libraries
import pandas as pd
import numpy as np

In [38]:
#load tic data
tic_data = pd.read_csv(r'tic_extract_20250213.csv')

In [39]:
df_tic = tic_data.copy()

In [40]:
df_tic.head()

,payer,network_name,network_id,network_year_month,network_region,code,code_type,ein,taxonomy_filtered_npi_list,modifier_list,billing_class,place_of_service_list,negotiation_type,arrangement,rate,cms_baseline_schedule,cms_baseline_rate
0,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,131740114,"1003990763,1023202793,1063525152,1063606739,10...",NaN,institutional,NaN,negotiated,ffs,15902.00,IPPS,6829.75
1,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,99283,CPT,131624096,"1003255670,1245759711,1487026522,1598095267,16...",NaN,professional,11,negotiated,ffs,123.86,PFS_NONFACILITY_1320201,76.89
2,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,"1700348620,1700892056,1922539964,1942685292",NaN,professional,11,negotiated,ffs,993.92,PFS_NONFACILITY_1320202,424.76
3,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,133971298,"1245635200,1437523537,1528013695,1528432622,15...",NaN,institutional,NaN,negotiated,ffs,27924.63,IPPS,6829.75
4,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,1346697315,NaN,professional,11,negotiated,ffs,849.63,PFS_NONFACILITY_1320203,391.85


In [41]:
#create tic key colmn
# payer key
df_tic['key_tic_payer'] = df_tic['payer'].str[:3].str.lower()

In [42]:
# hospital ein key
df_tic['key_tic_ein'] = df_tic['ein']

In [43]:
# procedure code key
df_tic['key_tic_code']= df_tic['code']

In [44]:
# group the faility and non-facility schedules
df_tic['cleaned_cms_baseline_schedule'] = np.where(df_tic['cms_baseline_schedule'].str.contains('_FACILITY_'), 'FACILITY',
                                                    np.where(df_tic['cms_baseline_schedule'].str.contains('_NONFACILITY_'), 'NON-FACILITY',
                                                             np.where(df_tic['cms_baseline_schedule'].str.contains('_NPA'), 'NPA',
                                                                      df_tic['cms_baseline_schedule'])))

In [45]:
# For the flatenned dataset: negotiation type and cleaned_cms_baseline_schedule
df_tic['key_tic_negotiation_type'] = df_tic['negotiation_type'] + '_' + df_tic['cleaned_cms_baseline_schedule']

In [46]:
df_tic_inter = df_tic[['key_tic_payer', 'key_tic_ein', 'key_tic_code', 'key_tic_negotiation_type','rate']]


In [47]:
# have key_tic_negotiation_type values as columns and median of rate as values
df_tic_pivot = df_tic_inter.pivot_table(index=['key_tic_payer', 'key_tic_ein', 'key_tic_code'], columns='key_tic_negotiation_type', values='rate', aggfunc='median').reset_index()

In [48]:
df_tic_pivot.head()

key_tic_negotiation_type,key_tic_payer,key_tic_ein,key_tic_code,fee schedule_FACILITY,fee schedule_IPPS,fee schedule_NON-FACILITY,fee schedule_OPPS,negotiated_FACILITY,negotiated_IPPS,negotiated_NON-FACILITY,negotiated_OPPS,percentage_NON-FACILITY,percentage_OPPS
0,aet,131624096,872,NaN,NaN,NaN,NaN,NaN,13365.0,NaN,NaN,NaN,NaN
1,aet,131624096,43239,168.25,NaN,453.640,NaN,4465.500,NaN,NaN,4465.5,NaN,NaN
2,aet,131624096,99283,NaN,NaN,61.835,NaN,NaN,NaN,NaN,NaN,NaN,70.0
3,aet,131740114,43239,NaN,NaN,NaN,NaN,335.145,NaN,453.63,11000.0,70.0,NaN
4,aet,131740114,99283,NaN,NaN,NaN,NaN,267.000,NaN,66.00,NaN,70.0,NaN


In [49]:
#load hpt data
hpt_data = pd.read_csv(r'hpt_extract_20250213.csv')

In [50]:
df_hpt = hpt_data.copy()

In [51]:
df_hpt.head()

,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer_name,plan_name,code_type,raw_code,...,modifiers,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_negotiated_percentage,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,additional_generic_notes
0,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Medicare,CPT,99283,...,NaN,NaN,NaN,323.34,NaN,83.78,1009.22,fee schedule,NaN,NaN
1,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,HealthFirst,Commercial Enrollees,CPT,43239,...,NaN,NaN,NaN,1037.65,NaN,165.40,3206.34,fee schedule,NaN,NaN
2,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Commercial,CPT,43239,...,NaN,NaN,NaN,1246.73,NaN,1246.73,1394.79,fee schedule,NaN,NaN
3,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Cigna,LocalPlus,CPT,99283,...,NaN,3744.0,2433.6,1797.00,NaN,225.00,1797.00,other,NaN,per visit
4,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Oscar,Medicare,CPT,43239,...,NaN,NaN,NaN,1037.65,NaN,141.77,1815.10,fee schedule,NaN,NaN


In [52]:
# create hpt keys for:
# hospital ein - by parsing the first 9 numeric characters from the source file name
df_hpt['key_hpt_ein'] = df_hpt['source_file_name'].str.replace('-', '').str[:9].astype(int)

In [53]:
# code key - cleaning up of stray strings
df_hpt['key_hpt_code'] = df_hpt['raw_code'].str.extract('(\d+)').astype(int)

In [54]:
# payer key - grouping of united, cigna and aetna aliases
df_hpt['key_hpt_payer'] = np.where(df_hpt['payer_name'].str.lower().str.contains('united'), 'uni',
                                   np.where(df_hpt['payer_name'].str.lower().str.contains('uhc'), 'uni',
                                            np.where(df_hpt['payer_name'].str.lower().str.contains('cigna'), 'cig',
                                                     np.where(df_hpt['payer_name'].str.lower().str.contains('aetna'), 'aet',
                                                              'Out of Scope'))))

In [55]:
# create the final merged data frame
df_merged = pd.merge(df_hpt, df_tic_pivot, how='left', left_on=['key_hpt_ein', 'key_hpt_code', 'key_hpt_payer'],
                     right_on=['key_tic_ein', 'key_tic_code', 'key_tic_payer'])

In [56]:
df_merged.head()

,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer_name,plan_name,code_type,raw_code,...,fee schedule_FACILITY,fee schedule_IPPS,fee schedule_NON-FACILITY,fee schedule_OPPS,negotiated_FACILITY,negotiated_IPPS,negotiated_NON-FACILITY,negotiated_OPPS,percentage_NON-FACILITY,percentage_OPPS
0,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Medicare,CPT,99283,...,NaN,NaN,NaN,NaN,267.000,NaN,66.00,NaN,70.0,NaN
1,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,HealthFirst,Commercial Enrollees,CPT,43239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Commercial,CPT,43239,...,NaN,NaN,NaN,NaN,335.145,NaN,453.63,11000.0,70.0,NaN
3,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Cigna,LocalPlus,CPT,99283,...,NaN,NaN,253.26,1902.0,NaN,NaN,NaN,NaN,NaN,NaN
4,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Oscar,Medicare,CPT,43239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
#write to csv
df_merged.to_csv(r'merged_data.csv', index=False)
